Importamos las librerías necesarias para el proceso de extracción, transformación y carga y luego extraemos los csv correspondientes.

In [2]:
import pandas as pd
from deep_translator import GoogleTranslator
from googletrans import Translator
import emoji
import re

Leemos los dataframes.

In [3]:
Closed_deals = pd.read_csv("Closed_deals.csv")
Customers = pd.read_csv("Customers.csv")
Geolocation = pd.read_csv("Geolocation.csv")
Marketing = pd.read_csv("Marketing_qualified_leads.csv")
Order_items = pd.read_csv("Order_items.csv")
Order_payments = pd.read_csv("Order_payments.csv")
Order_reviews = pd.read_csv("Order_reviews.csv")
Orders = pd.read_csv("Orders.csv")
Product_category_name_translation = pd.read_csv("Product_category_name_translation.csv")
Products = pd.read_csv("Products.csv")
Sellers = pd.read_csv("Sellers.csv")

Observamos que dentro del csv de closed deals hay columnas que no van a ser necesarias para nuestro análisis, por lo tanto las eliminamos.

In [4]:
Closed_deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     842 non-null    int64  
 1   mql_id                         842 non-null    object 
 2   seller_id                      842 non-null    object 
 3   sdr_id                         842 non-null    object 
 4   sr_id                          842 non-null    object 
 5   won_date                       842 non-null    object 
 6   business_segment               841 non-null    object 
 7   lead_type                      836 non-null    object 
 8   lead_behaviour_profile         665 non-null    object 
 9   has_company                    63 non-null     object 
 10  has_gtin                       64 non-null     object 
 11  average_stock                  66 non-null     object 
 12  business_type                  832 non-null    obj

In [5]:
Closed_deals = Closed_deals.drop(['has_company', 'has_gtin', 'average_stock', 'declared_product_catalog_size'], axis=1)

Ahora, a los valores nulos de las columnas cualitativas los rellenamos por "unknown" (desconocido), ya que va a resultar más adecuado para nuestro análisis.

In [6]:
Closed_deals['business_segment'].fillna('unknown', inplace=True)
Closed_deals['lead_type'].fillna('unknown', inplace=True)
Closed_deals['lead_behaviour_profile'].fillna('unknown', inplace=True)
Closed_deals['business_type'].fillna('unknown', inplace=True)

Dentro del csv 'Geolocation', creamos a una columna nueva donde nos ubique a qué región pertenece el Estado.

In [21]:
Geolocation['geolocation_state'].unique()

array(['SP', 'RN', 'AC', 'RJ', 'ES', 'MG', 'BA', 'SE', 'PE', 'AL', 'PB',
       'CE', 'PI', 'MA', 'PA', 'AP', 'AM', 'RR', 'DF', 'GO', 'RO', 'TO',
       'MT', 'MS', 'RS', 'PR', 'SC'], dtype=object)

In [7]:
Region={'PR':'Sur','RS':'Sur','SC':'Sur','SP':'Sudeste','MG':'Sudeste','RJ':'Sudeste','ES':'Sudeste','MT':'Centro oeste', 'MS':'Centro oeste', 'GO':'Centro oeste' , 'DF':'Centro oeste',
 'AC':'Norte', 'AP':'Norte','AM':'Norte','PA':'Norte', 'RO':'Norte','RR':'Norte', 'TO':'Norte', 	
'AL':'Nordeste','BA':'Nordeste','CE':'Nordeste','MA':'Nordeste','PB':'Nordeste','PI':'Nordeste',
'PE':'Nordeste','RN':'Nordeste','SE':'Nordeste',}

def get_region(state):
    return Region[state]

Geolocation['geolocation_region'] = Geolocation['geolocation_state'].apply(get_region)

In [8]:
Geolocation

,Unnamed: 0,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,geolocation_region
0,0,1037,-23.545621,-46.639292,sao paulo,SP,Sudeste
1,1,1046,-23.546081,-46.644820,sao paulo,SP,Sudeste
2,2,1046,-23.546129,-46.642951,sao paulo,SP,Sudeste
3,3,1041,-23.544392,-46.639499,sao paulo,SP,Sudeste
4,4,1035,-23.541578,-46.641607,sao paulo,SP,Sudeste
...,...,...,...,...,...,...,...
1000158,1000158,99950,-28.068639,-52.010705,tapejara,RS,Sur
1000159,1000159,99900,-27.877125,-52.224882,getulio vargas,RS,Sur
1000160,1000160,99950,-28.071855,-52.014716,tapejara,RS,Sur
1000161,1000161,99980,-28.388932,-51.846871,david canabarro,RS,Sur


Eliminación de datos duplicados.

In [9]:
Closed_deals = Closed_deals.drop_duplicates()
Customers = Customers.drop_duplicates()
Geolocation = Geolocation.drop_duplicates()
Marketing = Marketing.drop_duplicates()
Order_items = Order_items.drop_duplicates()
Order_payments = Order_payments.drop_duplicates()
Order_reviews = Order_reviews.drop_duplicates()
Orders = Orders.drop_duplicates()
Product_category_name_translation = Product_category_name_translation.drop_duplicates()
Products = Products.drop_duplicates()
Sellers = Sellers.drop_duplicates()

Cambiamos el caracter ã, por a, dentro del dataframe geolocation, ya que la misma ciudad aparecía escrita de dos maneras diferentes.

In [10]:
Geolocation['geolocation_city']= Geolocation['geolocation_city'].str.replace('ã','a')

Creamos una nueva columna en español para las sucursales latinoamericanas.

In [11]:
#le aplicamos la traducción a la columna product_category_name_english
translator = GoogleTranslator(source="auto", target="es")
Product_category_name_translation['product_category_name_spanish'] = Product_category_name_translation.product_category_name_english.apply(translator.translate)

print(Product_category_name_translation)

    Unnamed: 0          product_category_name product_category_name_english  \
0            0                   beleza_saude                 health_beauty   
1            1         informatica_acessorios         computers_accessories   
2            2                     automotivo                          auto   
3            3                cama_mesa_banho                bed_bath_table   
4            4               moveis_decoracao               furniture_decor   
..         ...                            ...                           ...   
66          66                         flores                       flowers   
67          67             artes_e_artesanato         arts_and_craftmanship   
68          68                fraldas_higiene           diapers_and_hygiene   
69          69  fashion_roupa_infanto_juvenil     fashion_childrens_clothes   
70          70             seguros_e_servicos         security_and_services   

   product_category_name_spanish  
0               

Normalizamos la columna Order_reviews, agregando "sin comentarios" y "sin título" en los valores nulos de las columnas de mensaje y título. Debajo, aplicamos una función para eliminar los emojis de los comentarios, ya que vamos a necesitar las columnas para análisis posteriores, y teniendo los emojis no podemos traducirlas.

In [12]:
Order_reviews['review_comment_message'].fillna('sem_comentarios',inplace=True)
Order_reviews['review_comment_title'].fillna('sem_titulo',inplace=True)

In [13]:
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)

def strip_emoji(text):
    return RE_EMOJI.sub(r'', text)

print(strip_emoji('🙄🤔as df'))

for i in range(0,100):
    Order_reviews['review_comment_message'][i]=strip_emoji(Order_reviews['review_comment_message'][i])
    Order_reviews['review_comment_title'][i]=strip_emoji(Order_reviews['review_comment_title'][i])

as df


C:\Users\Luis Castro\AppData\Local\Temp\ipykernel_4760\3017088374.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Order_reviews['review_comment_message'][i]=strip_emoji(Order_reviews['review_comment_message'][i])
C:\Users\Luis Castro\AppData\Local\Temp\ipykernel_4760\3017088374.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Order_reviews['review_comment_title'][i]=strip_emoji(Order_reviews['review_comment_title'][i])


Normalización de la tabla Order_reviews

In [14]:
Order_reviews['review_comment_message'] = Order_reviews['review_comment_message'].str.replace('\r\n',' ')
Order_reviews['review_comment_message'] = Order_reviews['review_comment_message'].apply(lambda text: emoji.demojize(text, delimiters=("", "")))
Order_reviews['review_comment_message'] = Order_reviews['review_comment_message'].str.replace(';',',')
Order_reviews['review_comment_message'] = Order_reviews['review_comment_message'].str.replace('"','')
Order_reviews['review_comment_message'] = Order_reviews['review_comment_message'].str.replace('\\','')
Order_reviews['review_comment_title'] = Order_reviews['review_comment_title'].str.replace('\r\n',' ')  
Order_reviews['review_comment_title'] = Order_reviews['review_comment_title'].apply(lambda text: emoji.demojize(text, delimiters=("", "")))
Order_reviews['review_comment_title'] = Order_reviews['review_comment_title'].str.replace(';',',')
Order_reviews['review_comment_title'] = Order_reviews['review_comment_title'].str.replace('"','')
Order_reviews['review_comment_title'] = Order_reviews['review_comment_title'].str.replace('\\','')

C:\Users\Luis Castro\AppData\Local\Temp\ipykernel_4760\1811077054.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Order_reviews['review_comment_message'] = Order_reviews['review_comment_message'].str.replace('\\','')
C:\Users\Luis Castro\AppData\Local\Temp\ipykernel_4760\1811077054.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Order_reviews['review_comment_title'] = Order_reviews['review_comment_title'].str.replace('\\','')
